<a href="https://colab.research.google.com/github/avinash064/Avinash_kashyap_21064/blob/main/21064_Avinashkashyap_nlpassignment4_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### NLP Assignment-2

Load data

In [2]:
!wget https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/main/train_split.csv
!wget https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/main/test_split.csv

--2024-11-15 17:10:26--  https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/main/train_split.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 144474 (141K) [text/plain]
Saving to: ‘train_split.csv’

train_split.csv     100%[===================>] 141.09K  --.-KB/s    in 0.1s    

2024-11-15 17:10:27 (1.24 MB/s) - ‘train_split.csv’ saved [144474/144474]

--2024-11-15 17:10:27--  https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/main/test_split.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35259 (34K) 

In [3]:
import pandas as pd

# Load the training DataFrame
train_data = pd.read_csv('/content/train_split.csv')
test_data = pd.read_csv('/content/test_split.csv')
print(test_data.head())

                                                text  Joy  Fear  Anger  \
0  And they probably knew exactly what your dad saw.    0     1      0   
1  My neck still hurts while I'm working, and I w...    0     1      1   
2  and Christian fish bumper stickers, it caught ...    1     0      0   
3                               It felt so unreal...    0     1      0   
4  ' To be honest I don't know why I set out into...    0     1      0   

   Sadness  Surprise  
0        0         1  
1        0         0  
2        0         1  
3        0         1  
4        0         0  


In [4]:
!pip install transformers
!pip install datasets
!pip install torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [5]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, RobertaTokenizer, RobertaForSequenceClassification
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import AlbertTokenizer, AlbertForSequenceClassification
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import AdamW
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
from datasets import Dataset

In [11]:
# Define a custom Dataset class
class EmotionDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = self.texts[item]
        label = self.labels[item]

        encoding = self.tokenizer.encode_plus(
            text,
            max_length=self.max_len,
            add_special_tokens=True,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.float)
        }

In [12]:
tokenizer_bert = BertTokenizer.from_pretrained('bert-base-uncased')
model_bert = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)

tokenizer_roberta = RobertaTokenizer.from_pretrained('roberta-base')
model_roberta = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=5)

tokenizer_distilbert = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model_distilbert = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=5)

tokenizer_albert = AlbertTokenizer.from_pretrained('albert-base-v2')
model_albert = AlbertForSequenceClassification.from_pretrained('albert-base-v2', num_labels=5)



model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
model_bert.classifier = torch.nn.Linear(model_bert.config.hidden_size, 5)
model_roberta.classifier = torch.nn.Linear(model_roberta.config.hidden_size, 5)
model_distilbert.classifier = torch.nn.Linear(model_distilbert.config.hidden_size, 5)
model_albert.classifier = torch.nn.Linear(model_albert.config.hidden_size, 5)



In [14]:
test_texts = test_data['text'].tolist()
test_labels = test_data.drop(columns=['text']).values  # Assuming the labels are in separate columns

max_len = 128  # Set a maximum length for the sequences


In [15]:
# Prepare the test dataset and dataloader
test_dataset = EmotionDataset(test_texts, test_labels, tokenizer_bert, max_len)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [16]:
def predict(model, tokenizer, dataloader):
    model.eval()  # Set model to evaluation mode
    predictions = []
    true_labels = []

    with torch.no_grad():  # No need to calculate gradients during inference
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            sigmoid_output = torch.sigmoid(logits)  # Apply sigmoid to get probability scores
            preds = (sigmoid_output > 0.5).int()  # Convert to 0 or 1 predictions based on 0.5 threshold

            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    return true_labels, predictions


In [17]:
"""def evaluate_model(model, tokenizer, dataloader):
    true_labels, predictions = predict(model, tokenizer, dataloader)
    # Flatten lists and calculate classification report
    report = classification_report(true_labels, predictions, target_names=["joy", "sadness", "fear", "anger", "surprise"], output_dict=True)
    return report

# Evaluate the models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move models to the device (GPU or CPU)
model_bert.to(device)
# model_roberta.to(device)
model_distilbert.to(device)
model_albert.to(device)

# Get evaluation reports for each model
report_bert = evaluate_model(model_bert, tokenizer_bert, test_dataloader)
# report_roberta = evaluate_model(model_roberta, tokenizer_roberta, test_dataloader)
report_distilbert = evaluate_model(model_distilbert, tokenizer_distilbert, test_dataloader)
report_albert = evaluate_model(model_albert, tokenizer_albert, test_dataloader)"""

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classifi

In [18]:
def print_classification_report(report, model_name):
    # Convert the classification report to a pandas DataFrame
    report_df = pd.DataFrame(report).T  # Transpose to make the categories rows

    # Add a column for the model name
    report_df['model'] = model_name

    # Print the formatted per-class report
    print(f"\n{model_name} Classification Report:")
    print(report_df)

    # Extract overall scores: micro avg, macro avg, weighted avg, samples avg
    overall_scores = {
        "micro avg": report['micro avg'],
        "macro avg": report['macro avg'],
        "weighted avg": report['weighted avg'],
        "samples avg": report['samples avg']
    }

    # Print the overall scores
    print("\nOverall Scores:")
    for key, value in overall_scores.items():
        print(f"{key.capitalize()}: {value['f1-score']:.4f}")

In [19]:
# Print the classification report for BERT
print_classification_report(report_bert, "BERT")

# Print the classification report for DistilBERT
print_classification_report(report_distilbert, "DistilBERT")

# Print the classification report for ALBERT
print_classification_report(report_albert, "ALBERT")



BERT Classification Report:
              precision    recall  f1-score  support model
joy            0.250000  0.010638  0.020408     94.0  BERT
sadness        0.580000  1.000000  0.734177    232.0  BERT
fear           0.500000  0.038462  0.071429     52.0  BERT
anger          0.301170  0.817460  0.440171    126.0  BERT
surprise       0.409091  0.072581  0.123288    124.0  BERT
micro avg      0.449482  0.552548  0.495714    628.0  BERT
macro avg      0.408052  0.387828  0.277895    628.0  BERT
weighted avg   0.434291  0.552548  0.392852    628.0  BERT
samples avg    0.459792  0.495208  0.449060    628.0  BERT

Overall Scores:
Micro avg: 0.4957
Macro avg: 0.2779
Weighted avg: 0.3929
Samples avg: 0.4491

DistilBERT Classification Report:
              precision    recall  f1-score  support       model
joy            0.000000  0.000000  0.000000     94.0  DistilBERT
sadness        0.568421  0.931034  0.705882    232.0  DistilBERT
fear           0.101010  0.192308  0.132450     52.0  Dis

In [20]:
model_distilbert_save_path = "/content/distilbert_trained_model.pth"
torch.save(model_distilbert.state_dict(), model_distilbert_save_path)
print(f"DistilBERT model saved to {model_distilbert_save_path}")


DistilBERT model saved to /content/distilbert_trained_model.pth


In [23]:
import shutil

  # Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Specify the source (model) and destination paths
source_path = "/content/distilbert_trained_model.pth"
destination_path = "/content/drive/My Drive/distilbert_trained_model.pth"

# Step 3: Copy the file to Google Drive
shutil.copy(source_path, destination_path)

print(f"Model saved to {destination_path}")

Mounted at /content/drive
Model saved to /content/drive/My Drive/distilbert_trained_model.pth


In [24]:
import gdown

# File ID and destination path
file_id = "1EpnFlIl_0_x8dMMMzahvi5l8HHgBBOh0"
destination = "/content/downloaded_file"

# Construct the gdown URL
gdown_url = f"https://drive.google.com/uc?id={file_id}"

# Download the file
gdown.download(gdown_url, destination, quiet=False)

# Verify the download
!ls -lh /content/downloaded_file


Downloading...
From (original): https://drive.google.com/uc?id=1EpnFlIl_0_x8dMMMzahvi5l8HHgBBOh0
From (redirected): https://drive.google.com/uc?id=1EpnFlIl_0_x8dMMMzahvi5l8HHgBBOh0&confirm=t&uuid=ae5ffc12-f4df-4db1-b69e-57375aae950f
To: /content/downloaded_file
100%|██████████| 268M/268M [00:06<00:00, 38.7MB/s]


-rw-r--r-- 1 root root 256M Nov 15 17:33 /content/downloaded_file
